# Countour plot of Helium energy 

Let's make countor plot of Helium energy with varying parameters 

In [23]:
import sys 
import matplotlib.pyplot as plt 
import numpy as np 
import numexpr as ne
import math 


from multiprocessing import Pool 

In [24]:
%run functions_module.ipynb 
%run auxiliaryFunctions_noMath_module.ipynb 
%run gradientDescent_module.ipynb 

# Preparing for countor plot 

## Load local energy 

In [25]:
localEnergyExprPath = "localEnergy_Helium"
# default folder to the Python-compatible formulae 
defaultFolderPathForFormulae = sys.path[0]+ "/"+ "Formulae_PythonCompatible/"    

In [26]:
localEnergyExpr = readFileToString(defaultFolderPathForFormulae + localEnergyExprPath)
localEnergyExpr[:50]
# localEnergyExpr = localEnergyExpr.replace('math.', '')    # remove all math-module functions 

'math.exp(A3*math.sqrt(x1**2 + y1**2 + z1**2) + A3*'

## Define local energy function 

In [27]:
# find coordinates and parameters
coordinates, parameterStringList  = findVariables(localEnergyExpr)

# regrouping coordinates 
coordinates = regroupToCoordinateTriple_findVariables(coordinates)


# for functions usage, define list of ordered variables and parameters
mapping =  coordinates + parameterStringList  # global variable: used in probability and local energy functions definition


In [28]:
mapping 

['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'A2', 'A3']

In [29]:



def localEnergyFunc(R: np.ndarray, parameters: list, expr: str = localEnergyExpr, mapping: list = mapping ) -> float: 
    '''
    
    H*psi/psi. Local energy.  Dependent on configuration and parameters. 
    
    Parameters
    ----------
        R: current configuration
        
        parameters: list of parameters as numbers 
        
        expr: formula string 
        
        mapping: ordered coordinates and parameters in a list: 
            ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2']
            
    Return
    ------
        Local energy evaluated at certain configuration and parameters' values
    
    '''
    
    
    R_flattened = R.flatten() # make current configuration to vector form 
    R_and_parameters = np.append(R_flattened, parameters) # ordered coordinates, and parameters at the tail of the vector 
    
    
    if len(R_and_parameters) != len(mapping): 
        raise Exception("R and parameters not same length as that of mapping!")
    
    
    # mapping dictionary for evaluation of the expression
    localDict = dict(zip(mapping, R_and_parameters))

    
    
    
    
    return eval(expr, {'math': math}, localDict)
    
    
    

## Load in pre-generated metropolis samples 

In [30]:
with open('metropolis_samples_opt_small.npy', 'rb') as file:
    metropolisSamples_Opt = np.load(file)

In [31]:
metropolisSamples_Opt.shape 

(2, 20, 3)

# Countour plot 

In [38]:
a, b = np.linspace(-1, 5, 6), np.linspace(-1, 5, 6)

A, B = np.meshgrid(a,b)

# initiate energies at different point in parameters' space 
energies_array = np.zeros(A.shape )


In [39]:
# index list combinations for computing energies with differring parameters
indList = []
for rowInd in range(A.shape[0]): 
    for columnInd in range(A.shape[1]): 
        indList.append((rowInd, columnInd))
        

In [40]:


def energyArraysFunc(rowInd: int, columnInd: int): 
        '''
        Function for parallel computation of energy values with varying parameters A and B 

        Parameters
        ---------
        rowInd: row index 

        columnInd: column index 

        Returns
        ------
        energy result and tuple referring to the location of the energy value using indices. 

        '''

        result  = monteCarloIntegrationFunction(samples=metropolisSamples_Opt, 
                                  localEnergyFunction=localEnergyFunc, 
                                 params = [A[rowInd, columnInd] , B[rowInd, columnInd]])

        return result, (rowInd, columnInd)
        
        

In [41]:



with Pool() as pool: 
    for result in pool.starmap(energyArraysFunc, indList): 
        energies_array[result[1]] = result[0]
        



ValueError: math domain error

In [ ]:
plt.cm.inferno

In [ ]:
plt.contourf(A, B, energies_array, cmap= 'inferno')
plt.xlabel('A')
plt.ylabel('B')
plt.colorbar()